In [1]:
import pandas as pd
import numpy as np
import time
from binance.client import Client
from datetime import datetime

In [2]:
api_key = 'bNpXIJwWOwPitmuUoKZsyuPV8lI1mcJV4iljO5gUrV5oShZzT98qaYxn7vVyIlLJ'
api_secret = '5oO1t4GwVq9Q3ixm2aequZije30NkyA2Pv6ZQAIB2za8TovVXbWWS3HxD9uBv2ZX'

In [3]:
client = Client(api_key, api_secret)

In [4]:
def fetch_pepe_1m_data(symbol='PEPEUSDT', 
                       start_date='2023-05-05 16:00:00',  # 请根据PEPE上线实际时间调整
                       end_date='2024-12-03 00:00:00'):
    """
    获取指定交易对从给定开始日期以来的1分钟级数据（K线数据）。
    参数:
    - symbol: 交易对，默认 'PEPEUSDT'
    - start_date: 起始时间，如 '2023-05-05 16:00:00'（PEPE上线时间）
    - end_date: 结束时间，如 '2024-12-03 00:00:00'
    """
    # 将日期转换为毫秒级时间戳
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_time = int(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)

    # 使用 get_historical_klines 获取历史K线数据
    # get_historical_klines 会自动进行分段请求直至获取全部数据（在较长区间数据量较大时可能需要等待一段时间）
    klines = client.get_historical_klines(symbol, '1m', start_time, end_time)

    # 整理数据并转换为DataFrame
    data = []
    for k in klines:
        # kline 数据格式：
        # [
        #   0: 开盘时间(open time, in ms),
        #   1: 开盘价(open),
        #   2: 最高价(high),
        #   3: 最低价(low),
        #   4: 收盘价(close),
        #   5: 成交量(volume),
        #   6: 收盘时间(close time),
        #   7: 成交额(quote asset volume),
        #   8: 成交笔数(number of trades),
        #   9: 主动买入成交量(taker buy base asset volume),
        #   10: 主动买入成交额(taker buy quote asset volume),
        #   11: 忽略(ignore)
        # ]
        data.append({
            'open_time': pd.to_datetime(k[0], unit='ms'),
            'open': float(k[1]),
            'high': float(k[2]),
            'low': float(k[3]),
            'close': float(k[4]),
            'volume': float(k[5]),
            'close_time': pd.to_datetime(k[6], unit='ms'),
            'quote_asset_volume': float(k[7]),
            'number_of_trades': int(k[8]),
            'taker_buy_base_asset_volume': float(k[9]),
            'taker_buy_quote_asset_volume': float(k[10])
        })

    df = pd.DataFrame(data)
    return df

In [6]:
from datetime import datetime

start_date = '2015-01-01 00:00:00'
end_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 获取PEPE/USDT的1分钟级K线数据
df_1m = fetch_pepe_1m_data('PEPEUSDT', start_date, end_date)


In [7]:
df_1m = df_1m.drop(['close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'], axis=1)

In [8]:
df_1m['date'] = df_1m['open_time']
df_1m = df_1m.drop(['open_time'], axis=1)

In [9]:
df_1m = df_1m.set_index('date')

In [10]:
df_1m['return'] = df_1m['close'].pct_change()

In [11]:
df_1m.to_csv('PEPE_1m_raw.csv')

In [1]:
import pandas as pd

In [2]:
df_1m = pd.read_csv('PEPE_1m_raw.csv')

In [4]:
import pandas_ta as ta

In [ ]:
my_strategy = ta.Strategy(
    name="My Custom 50 Indicators Strategy",
    ta=[
        {"kind": "rsi", "length": 14},
        {"kind": "rsi", "length": 7},
        {"kind": "stoch", "k":14, "d":3, "smooth":3},
        {"kind": "stochrsi", "length":14},
        {"kind": "macd", "fast":12, "slow":26, "signal":9},
        {"kind": "ppo", "fast":12, "slow":26, "signal":9},
        {"kind": "pvo", "fast":12, "slow":26, "signal":9},
        {"kind": "ao"},
        {"kind": "mfi", "length":14},
        {"kind": "cmf", "length":20},
        {"kind": "obv"},
        {"kind": "ad"},
        {"kind": "eom", "length":14},
        {"kind": "roc", "length":10},
        {"kind": "mom", "length":10},
        {"kind": "cci", "length":20},
        {"kind": "willr", "length":14},
        {"kind": "aroon", "length":14},
        {"kind": "dpo", "length":20},
        {"kind": "chop", "length":14},
        {"kind": "adx", "length":14},
        {"kind": "adx", "length":14},
        {"kind": "trix", "length":14},
        {"kind": "kst", "roc1":10, "roc2":15, "roc3":20, "roc4":30, "signal":9},
        {"kind": "tsi", "long":25, "short":13},
        {"kind": "fisher", "length":9},
        {"kind": "uo", "fast":7, "medium":14, "slow":28},
        {"kind": "rvi", "length":14},
        
        {"kind": "atr", "length":14},
        {"kind": "bbands", "length":20, "std":2},
        {"kind": "kc", "length":20},
        {"kind": "donchian", "lower_length":20, "upper_length":20},
        {"kind": "psar"},
        {"kind": "supertrend", "length":10, "multiplier":3.0},
        {"kind": "ichimoku"},  # 会生成多个列
        {"kind": "linreg", "length":14},
        
        {"kind": "ema", "length":200},
        {"kind": "ema", "length":50},
        {"kind": "ema", "length":21},
        {"kind": "sma", "length":200},
        {"kind": "sma", "length":50},
        {"kind": "sma", "length":21},
        {"kind": "dema", "length":14},
        {"kind": "tema", "length":14},
        {"kind": "wma", "length":14},
        {"kind": "hma", "length":14},
        {"kind": "t3", "length":14},
        
        {"kind": "corr", "length":20},  # 相关系数，观察价格相关性
    ]
)

# 使用自定义策略对数据进行技术指标计算
df_1m.ta.strategy(my_strategy)

AttributeError: 'AnalysisIndicators' object has no attribute 'dmi'

In [14]:
df_1m

,open,high,low,close,volume,return,SMA_10,SMA_20,SMA_50,SMA_200,VOL_SMA_20
date,,,,,,,,,,,
2023-05-05 18:00:00,0.000002,0.000004,0.000002,0.000003,2.528072e+12,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-05 18:01:00,0.000003,0.000004,0.000003,0.000004,1.283259e+12,0.088415,NaN,NaN,NaN,NaN,NaN
2023-05-05 18:02:00,0.000004,0.000004,0.000003,0.000004,5.440265e+11,-0.011204,NaN,NaN,NaN,NaN,NaN
2023-05-05 18:03:00,0.000004,0.000004,0.000003,0.000003,2.912850e+11,-0.025496,NaN,NaN,NaN,NaN,NaN
2023-05-05 18:04:00,0.000003,0.000003,0.000003,0.000003,4.250063e+11,-0.043605,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-06 12:33:00,0.000021,0.000021,0.000021,0.000021,2.158148e+10,-0.002383,0.000021,0.000021,0.000021,0.000021,1.077395e+10
2024-12-06 12:34:00,0.000021,0.000021,0.000021,0.000021,1.301954e+10,0.001433,0.000021,0.000021,0.000021,0.000021,1.118307e+10
2024-12-06 12:35:00,0.000021,0.000021,0.000021,0.000021,1.065016e+10,-0.000477,0.000021,0.000021,0.000021,0.000021,1.131294e+10
